# JODA 2022 Learning diary

-This notebook is available on: https://github.com/htaim/JODA2022/blob/620a8bcbdd31c540c7b59642ec6126cfa45628d4/Oppimispaivakirja.ipynb

## Week 1

-Created Git Repository: https://github.com/htaim/JODA2022

-The learning diary for this cource will be updated on that Git Repository once a week minimum.

-I created that repository and cloned it into my laptop in TUNI OneDrive folder, so there is a backup of backup with this implementation. Version control in Git is easy, and commits will be done often. This ensures agility with going back to older versions. Git is easy to use from command line. At the end of the course, the whole learning diary can be published for easy access with .zip export.

This week I participated in live lecture, and watched the demo session recording. I noticed that the course staff uses MacOS devices, and some of the installation steps could be explained a little more specified format. I think that is not relevant to only show "type pip install...", but explain the different installation options and how they differ from each other.

### Week's insights

1. Learned to created Git repo and improved in creating ssh keys and using command line
2. Learned from Jupyter Notebooks, how they are used, and how they work
3. Learned to use Python commands inside the Jupyter Notebook
4. Learned to use Markdown to format text in Jupyter Notebook
5. Learned to use basic Pandas functions

### Code demo

In [1]:
#Demo how to import data into this Python notebook, and do some basic analysis
#Code illustrated in this partially referenced from: https://github.com/InfoTUNI/joda2022/blob/master/koodiesimerkit/Session%201%20-%20Analysis%20of%20Transactions.ipynb
#Libraries can be imported as:
import pandas
import matplotlib.pyplot as plot
from matplotlib import style

In [9]:
#Data can be imported into pandas from url containing a csv file:
url = 'https://github.com/jodatut/2020/raw/master/koodiesimerkit/SalesJan2009.csv'

dataframe = pandas.read_csv(url)

#An overview of data and data types
print(dataframe.head())
print(dataframe.dtypes)

  Transaction_date   Product  Price Payment_Type               Name  \
0      1/2/09 6:17  Product1   1200   Mastercard           carolina   
1      1/2/09 4:53  Product1   1200         Visa             Betina   
2     1/2/09 13:08  Product1   1200   Mastercard  Federica e Andrea   
3     1/3/09 14:44  Product1   1200         Visa              Gouya   
4     1/4/09 12:56  Product2   3600         Visa            Gerd W    

                           City     State         Country Account_Created  \
0                      Basildon   England  United Kingdom     1/2/09 6:00   
1  Parkville                           MO   United States     1/2/09 4:42   
2  Astoria                             OR   United States    1/1/09 16:21   
3                        Echuca  Victoria       Australia   9/25/05 21:13   
4  Cahaba Heights                      AL   United States  11/15/08 15:47   

     Last_Login   Latitude   Longitude  
0   1/2/09 6:08  51.500000   -1.116667  
1   1/2/09 7:49  39.195000  

In [ ]:
#The datatypes can be changed:
dataframe['Transaction_date'] = pd.to_datetime(dataframe['Transaction_date'])
print(dataframe.dtypes)

### Learning outcomes

1. My goal is to take a step forward to data analytics, learn the latest and most important tools and improve myself as a data scientist.

2. I will be chasing the most advanced tools and methods in this course, but the highest course grade will not be my main goal.

3. I try to focus especially on the main assignment. I will use that as a part of my portfolio in CV.